# Guía 2: Gráficos en R parte 1
Computación 2, IES.
Profesor: Eduardo Jorquera, eduardo.jorquera@postgrado.uv.cl

## Manipulación de datos con R

Uno de los pasos que más consume tiempo en cualquier análisis de datos es la limpieza de éstos para que queden en un formato que permita el análisis. 

Para esto, necesitarás los siguientes paquetes instalados:

In [ ]:
pkgs <- c('reshape2', 'plyr', 'ggplot2', 'dplyr', 'data.table', 'Lahman')
install.packages(pkgs)

## Datos ordenados

Los datos crudos en el mundo real, están con frecuencia desordenados y cuentan con un formato bastante pobre. Más aún, pueden faltar detalles apropiados para el estudio que se quiere hacer. La corrección de estos datos puede transformarse en un ejercicio peligroso, ya que los datos originales serían sobre escritos y no habría froma de auditar este proceso o de reparar los errores cometidos al tomar/imputar los datos. Una buena práctica sería mantener los datos originales, y usar un código que los limpiara, arregle errores de imputación y guarde los datos limpios para futuros análisis. En esta sección aprenderás todo para ordenar los datos.

* Pregunta: Considere los siguientes datos. Cuántas variabes contiene este conjunto de datos?

| Hombres | mujeres |
-----|-------|---------|
Lesionado| 4     | 1       |
No lesionado| 2     | 5       | 


### ¿Qué son los datos ordenados?

Un conjunto de datos se dice que está ordenado si:
* Las observaciones están en filas
* Las variables están en columnas
* Está contenido en un solo dataset

### Explorando datos sucios

Veamos un ejemplo de cuasas comunes de suciedad en los datos con algunos pocos datasets.


#### Distribución de salario por religión

Nuestro primer dataset está basado en una encuesta hecha por *Pew Research* que examina la relación entre el salario y la afiliación religiosa.

Lee el dataset en R e inspecciona las primeras filas para ver si está todo ordenado:

In [31]:
pew <- read.delim(
  file = "http://stat405.had.co.nz/data/pew.txt",
  header = TRUE,
  stringsAsFactors = FALSE,
  check.names = F
)
head(pew)
write.table(pew,"pew.txt",sep=",",row.names=TRUE)

religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Agnostic,27,34,60,81,76,137,122,109,84,96
Atheist,12,27,37,52,35,70,73,59,74,76
Buddhist,27,21,30,34,33,58,62,39,53,54
Catholic,418,617,732,670,638,1116,949,792,633,1489
Don’t know/refused,15,14,15,11,10,35,21,17,18,116
Evangelical Prot,575,869,1064,982,881,1486,949,723,414,1529


#### Incidencia de tuberculosis

In [11]:
tb <- read.csv(
  file = "http://stat405.had.co.nz/data/tb.csv",
  header = TRUE, 
  stringsAsFactors = FALSE
)
head(tb)

iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,⋯,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
AD,1989,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AD,1990,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AD,1991,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AD,1992,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AD,1993,15,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AD,1994,24,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


#### Datos climáticos

In [13]:
weather <- read.delim(
 file = "http://stat405.had.co.nz/data/weather.txt",
 stringsAsFactors = FALSE
)
head(weather)

id,year,month,element,d1,d2,d3,d4,d5,d6,⋯,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<lgl>,<int>,<lgl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
MX000017004,2010,1,TMAX,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,278,NA
MX000017004,2010,1,TMIN,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,145,NA
MX000017004,2010,2,TMAX,NA,273,241,NA,NA,NA,⋯,NA,299,NA,NA,NA,NA,NA,NA,NA,NA
MX000017004,2010,2,TMIN,NA,144,144,NA,NA,NA,⋯,NA,107,NA,NA,NA,NA,NA,NA,NA,NA
MX000017004,2010,3,TMAX,NA,NA,NA,NA,321,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
MX000017004,2010,3,TMIN,NA,NA,NA,NA,142,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


#### Causas de suciedad en los datos

Hay varias características que pueden ser observadas en la práctica. Aquí hay algunos de los más observados patrones.
* El nombre de la columna son valores, no los nombres de las variables.
* Múltiples variables almacenadas en una sola columna.
* Las variables están almacenadas tanto en filas como en columnas.
* Múltiples tipos de unidades experimentales almacenadas en la misma tabla.
* Un tipo de unidad experimental almacenada en múltiples tablas.

## *Reshape*

Una forma de ordenar los datos, es cambiando la forma de todo el dataset; es decir, *reshape*. Cambiar la forma de los datos puede ayudar a ajustarlos a las tres reglas para tener datos ordenados. Si bien R por defecto tiene varias funciones que lo permiten, trabajaremos con la librería ``reshape2`` de Hadley Wickham, que provee un conjunto de funciones consistente para cambiar la forma de los datos.

### Ejemplo

En los términos más simples, remodelar (*reshape*) los datos es como hacer una tabla pivote en excel, donde cambias las columnas, las filas, y los valores. Empecemos ordenando los datos del dataset de pew.

In [32]:
pew <- read.delim(
  file = "pew.txt",
  header = TRUE,
  stringsAsFactors = FALSE,
  check.names = F, sep=","
)
head(pew)

religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Agnostic,27,34,60,81,76,137,122,109,84,96
Atheist,12,27,37,52,35,70,73,59,74,76
Buddhist,27,21,30,34,33,58,62,39,53,54
Catholic,418,617,732,670,638,1116,949,792,633,1489
Don’t know/refused,15,14,15,11,10,35,21,17,18,116
Evangelical Prot,575,869,1064,982,881,1486,949,723,414,1529


Podemos ordenar estos datos usando la función `melt`, alojada en el paquete `reshape2`.

In [35]:
library(reshape2)
pew_tidy <- melt(
  data = pew,
  id = "religion",
  variable.name = "income",
  value.name = "frequency"
)
head(pew_tidy)

religion,income,frequency
<chr>,<fct>,<int>
Agnostic,<$10k,27
Atheist,<$10k,12
Buddhist,<$10k,27
Catholic,<$10k,418
Don’t know/refused,<$10k,15
Evangelical Prot,<$10k,575


### Ejercicio 1

Ahora que sabes como usar `melt` en un dataframe, usa la misma idea para ordenar los daros de tuberculosis. Piensa en cómo puedes separar género y edad después de hacer `melt` en los datos.

### Ejercicio 2
Usa los mismos principios para los datos climáticos.